### Objectifs de ce notebook :
>Utilisation des données du site https://data.cityofchicago.org/ sur la criminalité ainsi que des données  socio-économiques du ministère de la Santé de Chicago https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2.
- Extraction des features
- Label ou taget (variables à prédire) : type de crime à prédire
- Implémentation d'algorithme de machine learning sur l'annés 2012 et application sur l'année 2013
- Visualisation des résultats

In [ ]:
import os
import sys
sys.path.append('../')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from yaml import load as yaml_load

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
import scipy
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
category = LabelEncoder()

In [ ]:
Path_Socio = "/home/ml/Documents/crimes_chigaco/data/raw/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv"
Path_Crime = "/home/ml/Documents/crimes_chigaco/data/raw/Crimes_-_2001_to_present.csv"
Path_Temp = "/home/ml/Documents/crimes_chigaco/data/raw/temperature.csv"
Path_sky = "/home/ml/Documents/crimes_chigaco/data/raw/weather_description.csv"

In [ ]:
df_temperature = pd.read_csv(Path_Temp, parse_dates=['datetime'])

In [ ]:
df_sky = pd.read_csv(Path_sky, parse_dates=['datetime'])

In [ ]:
df_sky.dropna(inplace=True)
df_sky =df_sky[['Chicago', 'datetime']]

In [ ]:
df_sky=pd.get_dummies(df_sky, columns=['Chicago'])

In [ ]:
df_sky

In [ ]:
df_temp = df_temperature[['datetime', 'Chicago']]

In [ ]:
df_temp = df_temp[('2013-01'<=df_temp['datetime']) & (df_temp['datetime']<'2014-01')]

In [ ]:
df_temp.rename(columns = {'Chicago': 'Temperature'}, inplace = True)

In [ ]:
df_temp['month'] = df_temp ['datetime'].dt.month
df_temp['day'] = df_temp['datetime'].dt.day
df_temp['hours'] = df_temp['datetime'].dt.hour

In [ ]:
df_temp = df_temp[['Temperature', 'month', 'day', 'hours']]

In [ ]:
df_Socio = pd.read_csv(Path_Socio)
df_Crime = pd.read_csv(Path_Crime, sep=';', parse_dates=['Date'])

In [ ]:
def set_param():
    
    # setup parameters for xgboost
    param = {}
    param['objective'] = 'multi:softmax'
    param['eta'] = 0.1
    param['silent'] = 0
    param['nthread'] = 4
    param['num_class'] = num_class
    param['eval_metric'] = 'mlogloss'
    param[' n_estimators'] = 300
    # Model complexity
    param['max_depth'] = 10 #set to 8
    param['min_child_weight'] = 1
    param['gamma'] = 0 
    param['reg_alfa'] = 0.05

    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8 #set to 1

    # Imbalanced data
    param['max_delta_step'] = 1
    
    return param

In [ ]:
def rename_columns_socio():
    """
    
    """
    return {
'Community Area Number':'community_area_number',
'COMMUNITY AREA NAME':'community_area_name', 
'PERCENT OF HOUSING CROWDED':'pct_housing_crowded', 
'PERCENT HOUSEHOLDS BELOW POVERTY':'pct_households_below_poverty', 
'PERCENT AGED 16+ UNEMPLOYED':'pct_age16_unemployed',
'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA':'pct_age25_no_highschool',
'PERCENT AGED UNDER 18 OR OVER 64': 'pct_not_working_age',
'per_capita_income':'per_capita_income',
'HARDSHIP INDEX' : 'hardship_index'}

def rename_columns_crimes():
    """
    
    """
    return {
        'ID': 'id',
        'Case Number': 'cas_number', 
        'Date':'date',
        'Block':'block', 
        'IUCR':'iucr', 
        'Primary Type':'primary_type',
        'Description':'description', 
        'Location Description':'location_description', 
        'Arrest':'arrest', 
        'Domestic':'domestic', 
        'Beat':'beat',
        'District':'district', 
        'Ward':'ward', 
        'Community Area': 'community_area_number', 
        'FBI Code':'fbi_code', 
        'X Coordinate':'x_coordinate',
        'Y Coordinate':'y_coordinate', 
        'Year':'year', 
        'Updated On':'updated_on', 
        'Latitude':'latitude', 
        'Longitude':'longitude',
        'Location':'location'   
    }

df_Socio.rename(columns=rename_columns_socio(), inplace=True)
df_Crime.rename(columns=rename_columns_crimes(), inplace=True)

In [ ]:
df_Crime_2013 = df_Crime[df_Crime['year']==2013]

In [ ]:
df_Crime_2013.sample(5)

In [ ]:
df_Crime_2012['extract_block'] = df_Crime_2012.block.apply(lambda x : re.findall(r"(\w+)$",  x)[0])

In [ ]:
df_Crime_2012['extract_block'].unique()

In [ ]:
df_Crime_2012.extract_block.value_counts()

In [ ]:
def isStreet(x):
    if x in ['ST', 'St', 'st']:
        return 1
    else:
        return 0

def isAV(x):
    if x in ['Ave', 'AV']:
        return 1
    else:
        return 0

def isBLVD(x):
    if x in ['BLVD']:
        return 1
    else:
        return 0

def isRD(x):
    if x in ['RD']:
        return 1
    else:
        return 0

def isPL(x):
    if x in ['PL', 'pl']:
        return 1
    else:
        return 0

def isBROADWAY(x):
    if x in ['BROADWAY', 'Broadway']:
        return 1
    else:
        return 0

def isPKWY(x):
    if x in ['PKWY','Pkwy']:
        return 1
    else:
        return 0    

In [ ]:
df_Crime_2012['isStreet'] = df_Crime_2012.extract_block.apply(lambda x: isStreet(x))
df_Crime_2012['isAV'] = df_Crime_2012.extract_block.apply(lambda x: isAV(x))
df_Crime_2012['isBLVD'] = df_Crime_2012.extract_block.apply(lambda x: isBLVD(x))
df_Crime_2012['isRD'] = df_Crime_2012.extract_block.apply(lambda x: isRD(x))
df_Crime_2012['isPL'] = df_Crime_2012.extract_block.apply(lambda x: isPL(x))
df_Crime_2012['isBROADWAY'] = df_Crime_2012.extract_block.apply(lambda x: isBROADWAY(x))
df_Crime_2012['isPKWY'] = df_Crime_2012.extract_block.apply(lambda x: isPKWY(x))

In [ ]:
list(df_Crime_2012.columns)

In [ ]:
df_Crime_2012.location_description.unique()

In [ ]:
len(df_Crime_2013.location_description.unique())

In [ ]:
list(df_Crime_2013.block.unique())

In [ ]:
import re

In [ ]:
print(re.findall(r"(\w+)$",  '008XX N OAKLEY BLVD'))

In [ ]:
## list of crime to predict
list_of_crimes = ['THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'BURGLARY','ASSAULT']
list_to_drop = ['id','location','cas_number','date','block','iucr','primary_type','updated_on',\
                                         'description','location_description','arrest','beat','ward','community_area_number','fbi_code']

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.figure(figsize=(15, 15))

In [ ]:
def extract_feature(year, df_C, df_S, list_of_crimes, list_to_drop):
    """
    this function extract features for machine learning algorithm
    """
    df = df_C[df_C['year']==year]
    df['extract_block'] = df.block.apply(lambda x : re.findall(r"(\w+)$",  x)[0])
    df = df[df.primary_type.isin(list_of_crimes)]
    df['isStreet'] = df.extract_block.apply(lambda x: isStreet(x))
    df['isAV'] = df.extract_block.apply(lambda x: isAV(x))
    df['isBLVD'] = df.extract_block.apply(lambda x: isBLVD(x))
    df['isRD'] = df.extract_block.apply(lambda x: isRD(x))
    df['isPL'] = df.extract_block.apply(lambda x: isPL(x))
    df['isBROADWAY'] = df.extract_block.apply(lambda x: isBROADWAY(x))
    df['isPKWY'] = df.extract_block.apply(lambda x: isPKWY(x))
    df.drop("extract_block", inplace=True, axis=1)
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hours'] = df['date'].dt.hour
    df['minutes'] = df['date'].dt.minute
    df['dayofweek'] = df['date'].apply(lambda x: dt.datetime.strftime(x, '%A'))
    df['XY'] = df.x_coordinate*df.y_coordinate 
    df['Category'] = category.fit_transform(df.primary_type)
    df = df[df.primary_type.isin(list_of_crimes)]
    df = pd.get_dummies(df, columns=['dayofweek'])
    df_crime_socio = pd.merge(df,df_S,on='community_area_number', how = 'left')
    print(list(df_crime_socio.domestic.unique()))
    df_crime_socio = pd.get_dummies(df_crime_socio, columns= ['domestic', 'community_area_name'])
    # df_crime_socio_ml = pd.get_dummies(data = df_crime_socio, columns=['community_area_name'])
    del df
    return df_crime_socio.drop(list_to_drop, axis=1), list(category.classes_)

In [ ]:
df_ml, class_names = extract_feature(2013, df_Crime, df_Socio, list_of_crimes, list_to_drop)

In [ ]:
df_ml.Category.value_counts()

In [ ]:
df_ml.sample(5)

In [ ]:
list(df_ml.columns)

In [ ]:
df_ml.shape

In [ ]:
df_ml_ = pd.merge(df_ml, df_temp, on = ['month', 'day', 'hours'], how ='left')

In [ ]:
df_ml_.shape

In [ ]:
'Chicago' in list(df_ml_.columns)

In [ ]:
list(df_Socio.columns)

### Implémentation d'algorithme de machine learning

In [ ]:
feature_cols = [x for x in df_ml if x!='Category']
X = df_ml[feature_cols]
y = df_ml['Category']
X_train, x_test, y_train, y_test = train_test_split(X, y,  test_size=0.3, random_state=42)

In [ ]:
list(df_ml.columns)

In [ ]:
df_ml['day'].sample(5)

In [ ]:
y.shape

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)
num_class = len(y.unique())
param = set_param()
watchlist = [(dtrain,'train'), (dtest, 'eval')]
num_round = 30
# Train XGBoost    
bst = xgb.train(param, dtrain, num_round, watchlist);

In [ ]:
num_class

In [ ]:
preds = bst.predict(dtest)

In [ ]:
from sklearn.metrics import precision_score
print(precision_score(y_test, preds, average='macro'))

In [ ]:
print(accuracy_score(y_test, preds))

In [ ]:
class_names

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_names = class_names
print(classification_report(y_test, preds, target_names=target_names))

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, preds) 
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figsize=(10, 10)
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figsize=(10, 10)
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

In [ ]:
from astral import Astral

In [ ]:
city_name = 'London'

In [ ]:
import datetime
a = Astral()
moon_phase = a.moon_phase(date=datetime.date(2018, 1, 1))

In [ ]:
print(moon_phase)

In [ ]:
import datetime
from astral import Astral
city_name = 'Chicago'
a = Astral()
a.solar_depression = 'civil'
city = a[city_name]

In [ ]:
moon_phase = a.moon_phase(date=datetime.date(2018, 1, 13))

In [ ]:
print(moon_phase)

In [ ]:
sun = city.sun(date=datetime.date(2009,4,22), local=True)

In [ ]:
d = (sun['sunset'] - sun['sunrise']).total_seconds()

In [ ]:
d

In [ ]:
d1 = df_sky['datetime'].iloc[1]

In [ ]:
d1

In [ ]:
pd.to_datetime(d1, format='%Y%m%d', errors='ignore')

In [ ]:
sun_ = city.sun(date=d1, local=True)

In [ ]:
d = (sun_['sunset'] - sun_['sunrise']).total_seconds()

In [ ]:
d